In [40]:
import numpy as np
import imagehash
import cv2
import os
import shutil
from PIL import Image
from tqdm import tqdm, tqdm_notebook

import matplotlib.pyplot as plt 

In [2]:
hash_funcs = [
        imagehash.average_hash,
        imagehash.phash,
        imagehash.dhash,
        imagehash.whash,
    ]

In [16]:
def is_eq(a, b, th=0.9):
    return float((a == b).astype(int).sum())/len(a) >= th


def make_hashes(src, funcs=hash_funcs):
    src_hashes = []
    print('make hashes')
    for image_file in tqdm_notebook(src):
        image = cv2.imread(image_file)
        image = Image.fromarray(image)
        src_hashes.append(np.array([f(image).hash for f in funcs]).reshape(256))
    return src_hashes


def make_files_lst(src):
    return [f'{src}/{file}' for file in os.listdir(src)]


def find_dup(src,  src_hashes=None, src_images=None, dst=None, dst_hashes=None, dst_images=None, funcs=hash_funcs):
    if src_images is None: 
        src_images = make_files_lst(src)
    if src_hashes is None:
        src_hashes = make_hashes(src_images, hash_funcs)
    dup_files_src = []
    dup_files_dst = []
    if dst is None:
        print('search for duplicates')
        for i in tqdm_notebook(range(len(src_hashes))):
            for j in range(i + 1, len(src_hashes), 1):
                if is_eq(src_hashes[i], src_hashes[j]):
                    dup_files_src.append(src_images[i])
                    dup_files_dst.append(src_images[j])        
    else:
        if dst_images is None:
            dst_images = make_files_lst(dst)
        if dst_hashes is None:
            dst_hashes = make_hashes(dst_images, hash_funcs)
        print('search for duplicates')
        for i, s in tqdm_notebook(enumerate(src_hashes)):
            for j, d in enumerate(dst_hashes):
                if is_eq(s, d):
                    dup_files_src.append(src_images[i])
                    dup_files_dst.append(dst_images[j])

    return dup_files_src, dup_files_dst

In [4]:
path = './'
path2 = './add_data'
path_train = f'{path}/train_images/'
path_dct = {0 : f'{path2}/train/cbb/',
            1 : f'{path2}/train/cbsd/',
            2 : f'{path2}/train/cgm/',
            3 : f'{path2}/train/cmd/',
            4 : f'{path2}/train/healthy/'}
path_extra = f'{path2}/extraimages/'

In [14]:
train_files = make_files_lst(path_train)

In [5]:
train_hashes = make_hashes(train_files)

  0%|                                                                                | 3/21397 [00:00<13:16, 26.86it/s]

make hashes


100%|████████████████████████████████████████████████████████████████████████████| 21397/21397 [08:39<00:00, 41.17it/s]
